In [36]:
import json
import pdb
import os
import nltk
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def get_tweet_text_from_json(file_path):
    events = []
    labels = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
            labels.append(value["label"])
        return events,labels
    
def get_tweet_text_from_json_unlabel(file_path):
    events = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
        return events
# a,a_labels = get_tweet_text_from_json("drive/FreeGPUnlp/train_negs_bbc_sport.json")
# b,b_labels = get_tweet_text_from_json("drive/FreeGPUnlp/train_negs_thegurdian_ausnews.json")
# c,c_labels = get_tweet_text_from_json("drive/FreeGPUnlp/train_negs_thegurdian_australian-immigration-and-asylum.json")
# d,d_labels = get_tweet_text_from_json("drive/FreeGPUnlp/train_negs_thegurdian_australian-politics.json")
# e,e_labels = get_tweet_text_from_json("drive/FreeGPUnlp/train_negs_thegurdian_uk-news.json")
# f,f_labels = get_tweet_text_from_json("drive/FreeGPUnlp/train_negs_thegurdian_world.json")
# g,g_labels = get_tweet_text_from_json("drive/FreeGPUnlp/train_negs_thegurdian.json")
# cor_info_events,cor_info_labels = a + b + c + d + e + f + g, a_labels + b_labels + c_labels + d_labels + e_labels + f_labels + g_labels
cor_info_events,cor_info_labels = get_tweet_text_from_json("train_negs.json")
misinfo_train_events,misinfo_train_labels = get_tweet_text_from_json("train.json")
dev_events,dev_labels = get_tweet_text_from_json("dev.json")
test_unlabel = get_tweet_text_from_json_unlabel("test-unlabelled.json")

In [43]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [73]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')

# stopwords = set(stopwords.words('english'))
wnl = WordNetLemmatizer()
tt = TweetTokenizer()
punc = punctuation + u'.,;《》？！@#￥%…&×（）——+【】{};；●，。&～、|\s:：'

stopwords = ['a',
 'about',
 'above',
 'after',
 'again',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'by',
 'can',
 'could',
 'd',
 'did',
 'do',
 'does',
 'doing',
 'down',
 'during',
 'each',
 'for',
 'from',
 'further',
 'had',
 'has',
 'have',
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'in',
 'into',
 'is',
 'it',
 "it's",
 'its',
 'itself',
 'll',
 'm',
 'ma',
 'me',
 'might',
 'more',
 'most',
 'must',
 'my',
 'myself',
 'now',
 'o',
 'of',
 'off',
 'on',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'shan',
 "shan't",
 'she',
 "she's",
 'some',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'we',
 'were',
 'what',
 'when',
 'where',
 'which',
 'while','who','whom','why','will','with','won',"won't",'wouldn',"wouldn't",'y','you',"you'd","you'll","you're","you've",'your','yours','yourself','yourselves']

from string import digits
def get_wordnet_pos(treebank_tag):

    if (treebank_tag.startswith('J'))|(treebank_tag =='ADJ'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif (treebank_tag.startswith('R'))|(treebank_tag=='ADV'):
        return wordnet.ADV
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN

def preprocess_drop_punc_evnets(events):
#     preprocess_drop_punc_evnets = events
    preprocess_drop_punc_evnets = []
#     pdb.set_trace()
    for event in events:
        event_re = re.sub(r"[{}]+".format(punc)," ",event)
        remove_digits = str.maketrans('', '', digits)
        preprocess_drop_punc_evnets.append(event_re.translate(remove_digits))
    return preprocess_drop_punc_evnets
  
def replace_name_with_placeholder(token):
    if token.ent_iob != 0 and token.label_ == "PERSON":
        return "YYYYYYYY"
    elif token.ent_iob != 0 and token.label_ == "GPE":
        return "XXXXXXXX"
    else:
        return token.string
    
def scrub(text):
    event = []
    doc = nlp(text)
    ents = [e.merge() for e in doc.ents if e.label_.lower() in ["person","gpe"]]
    for item in doc:
        if item in ents:
            pass
        else:
            event.append(item.text)
    return " ".join(event)
#     for ent in doc.ents:
#         ent.merge()
#     pdb.set_trace()
#     tokens = map(replace_name_with_placeholder, doc)
#     return "".join(tokens)

def ner(events):
    for i,event in enumerate(events):
        events[i] = scrub(event)
#         pdb.set_trace()
    return events

def preprocess_stopwords_lemma_evnets(events):
    preprocess_BOW_events = []
    preprocess_events = []
    preprocess_events_tokens = []
    events_ner = ner(events)
    pdb.set_trace()
    after_drop_punc_evnets = preprocess_drop_punc_evnets(events_ner)
    for event in after_drop_punc_evnets:
        dic_bow = defaultdict(int)
#         pdb.set_trace()
#         event_ner = scrub(event)
        event_not = re.sub("[.*]*n[‘|']t"," not",event)
        tokenize_words = tt.tokenize(event_not)
        words_pos = nltk.pos_tag(tokenize_words, tagset="universal")
#         pdb.set_trace()
        filter_event = [(w,pos) for (w,pos) in words_pos if w.lower() not in stopwords]
        lemma_event = []
        for (word,pos) in filter_event:
#             pdb.set_trace()
            word_lemma = wnl.lemmatize(word.lower(),get_wordnet_pos(pos))
            lemma_event.append(word_lemma)
            dic_bow[word_lemma] += 1
        preprocess_events_tokens.append(lemma_event,)
        preprocess_events.append(' '.join(lemma_event))
        preprocess_BOW_events.append(dic_bow)
    return preprocess_events_tokens,preprocess_events,preprocess_BOW_events

preprocess_train_events_tokens,preprocessed_train_events,preprocessed_train_BOW_events = preprocess_stopwords_lemma_evnets(misinfo_train_events)
preprocess_cor_events_tokens,preprocessed_cor_events,preprocessed_cor_BOW_events = preprocess_stopwords_lemma_evnets(cor_info_events)
preprocess_dev_events_tokens,preprocessed_dev_events,preprocessed_dev_BOW_events = preprocess_stopwords_lemma_evnets(dev_events)
preprocess_test_events_tokens,preprocessed_test,preprocessed_test_BOW = preprocess_stopwords_lemma_evnets(test_unlabel)

[nltk_data] Downloading package wordnet to /Users/liyi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/liyi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/liyi/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/liyi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


> <ipython-input-73-54ac14cbea15>(189)preprocess_stopwords_lemma_evnets()
-> after_drop_punc_evnets = preprocess_drop_punc_evnets(events_ner)
(Pdb) n
> <ipython-input-73-54ac14cbea15>(190)preprocess_stopwords_lemma_evnets()
-> for event in after_drop_punc_evnets:
(Pdb) n
> <ipython-input-73-54ac14cbea15>(191)preprocess_stopwords_lemma_evnets()
-> dic_bow = defaultdict(int)
(Pdb) n
> <ipython-input-73-54ac14cbea15>(194)preprocess_stopwords_lemma_evnets()
-> event_not = re.sub("[.*]*n[‘|']t"," not",event)
(Pdb) n
> <ipython-input-73-54ac14cbea15>(195)preprocess_stopwords_lemma_evnets()
-> tokenize_words = tt.tokenize(event_not)
(Pdb) event_not
'why flooding is not a sign of climate change Distinguished climate scientist writes In the context of climate change is what we are seeing in a new level of disaster which is becoming more common The flood disaster unfolding in is certainly very unusual But so are other natural weather disasters which have always occurred and always will occur Maj

In [74]:
train_labels = misinfo_train_labels + cor_info_labels

In [75]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def evaluate_result(groundtruth, prediction):
    prediction = prediction.tolist()
    for i,item in enumerate(prediction):
        if item == -1:
            prediction[i] = 0
    p, r, f, _ = precision_recall_fscore_support(groundtruth, prediction, pos_label=1, average="binary")
#     pdb.set_trace()
    # print("Performance on the positive class (documents with misinformation):")
    # print("Precision =", p)
    # print("Recall    =", r)
    # print("F1        =", f)
    # print(accuracy_score(groundtruth,prediction))
    return p,r,f,accuracy_score(groundtruth,prediction)
    
def transfer_output(preds):
    test_dic = {}
    for i,test_pre in enumerate(preds):

        if test_pre == -1:
            dic = {}
            dic['label'] = 0
    #         pdb.set_trace()
            test_dic['test-%s'%(i)] = dic
        elif test_pre == 0:
            dic = {}
    #         pdb.set_trace()
            dic['label'] = 0
            test_dic['test-%s'%(i)] = dic
        else:
            dic = {}
    #         pdb.set_trace()
            dic['label'] = 1
            test_dic['test-%s'%(i)] = dic
    jstr = json.dumps(test_dic,ensure_ascii = False)
    # print(jstr)
    with open('test-output.json','w') as f:
        f.write(jstr)

In [0]:
################################################ DictVectorizer ##################################################

In [0]:
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
vectorizer = DictVectorizer()

vector_train = vectorizer.fit_transform(preprocessed_train_BOW_events + preprocessed_cor_BOW_events)
vector_development = vectorizer.transform(preprocessed_dev_BOW_events)
vector_test = vectorizer.transform(preprocessed_test_BOW)

In [0]:
################################################ w2v_own ##################################################

In [0]:
import gensim
from gensim.models import Word2Vec
all_texts = preprocess_cor_events_tokens + preprocess_train_events_tokens + preprocess_dev_events_tokens + preprocess_test_events_tokens
word_model_skip_gram = Word2Vec(all_texts, min_count=1, size=60, window=8, iter=50, sg = 1)

In [0]:
################################################ w2v_google ##################################################

In [52]:
import gensim
from gensim.models import Word2Vec
import numpy as np
wv = gensim.models.KeyedVectors.load_word2vec_format("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

X_train_word_average = word_averaging_list(wv,preprocess_train_events_tokens + preprocess_cor_events_tokens)
X_dev_word_average = word_averaging_list(wv,preprocess_dev_events_tokens)
X_test_word_average = word_averaging_list(wv,preprocess_test_events_tokens)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
from collections import defaultdict
import numpy as np
class MeanEmbeddingVectorizer(object):

    def __init__(self, word_model):
        self.word_model = word_model
        self.vector_size = word_model.wv.vector_size

    def fit(self):  # comply with scikit-learn transformer requirement
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.


        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """
        mean = []
        for word in sent:
            if word in self.word_model.wv.vocab:
                mean.append(self.word_model.wv.get_vector(word))

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.

        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])
from sklearn.feature_extraction.text import TfidfVectorizer
class TfidfEmbeddingVectorizer(object):

    def __init__(self, word_model):

        self.word_model = word_model
        self.word_idf_weight = None
        self.vector_size = word_model.wv.vector_size

    def fit(self, docs):  # comply with scikit-learn transformer requirement
        """
        Fit in a list of docs, which had been preprocessed and tokenized,
        such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
        Then build up a tfidf model to compute each word's idf as its weight.
        Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
        :param
            pre_processed_docs: list of docs, which are tokenized
        :return:
            self
        """

        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer()
        tfidf.fit(text_docs)  # must be list of text string

        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)  # used as default value for defaultdict
        self.word_idf_weight = defaultdict(lambda: max_idf,[(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()])
        return self


    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector


    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """

        mean = []
        for word in sent:
            if word in self.word_model.wv.vocab:
                mean.append(self.word_model.wv.get_vector(word) * self.word_idf_weight[word])  # idf weighted

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

In [0]:
mean_vec_tr = MeanEmbeddingVectorizer(word_model_skip_gram)
train_vec = mean_vec_tr.transform(preprocess_train_events_tokens + preprocess_cor_events_tokens)
dev_vec = mean_vec_tr.transform(preprocess_dev_events_tokens)
test_vec = mean_vec_tr.transform(preprocess_test_events_tokens)

tfidf_vec_tr_skip_gram = TfidfEmbeddingVectorizer(word_model_skip_gram)
tfidf_vec_tr_skip_gram.fit(preprocess_train_events_tokens + preprocess_cor_events_tokens)  # fit tfidf model first
tfidf_train_vec_skip_gram = tfidf_vec_tr_skip_gram.transform(preprocess_train_events_tokens + preprocess_cor_events_tokens)
tfidf_dev_vec_skip_gram = tfidf_vec_tr_skip_gram.transform(preprocess_dev_events_tokens)
tfidf_test_vec_skip_gram = tfidf_vec_tr_skip_gram.transform(preprocess_test_events_tokens)

In [0]:
################################################ tfidf ##################################################

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvector = TfidfVectorizer(min_df=3,  
                              max_df=0.5,
                              max_features=None,                 
                              ngram_range=(1, 2), 
                              use_idf=True,
                              smooth_idf=True)
train_tfidfvector = tfidfvector.fit_transform(preprocessed_train_events + preprocessed_cor_events)
dev_tfidfvector = tfidfvector.transform(preprocessed_dev_events)
test_tfidfvector = tfidfvector.transform(preprocessed_test)

In [0]:
################################################ classic binary algrithom ##################################################

In [77]:
from sklearn.linear_model import LogisticRegression
def tune_log(train_feature,train_label,dev_feature,dev_label,test_feature):
    solvers = ["liblinear","lbfgs","newton-cg","sag"]
    penalties = ["l1","l2"]
    cs = [0.001,0.01,0.1,0.5,0.53,1,100, 1000]
    f_score = []
    precision = []
    recall = []
    parameter_save = []
    for solver in solvers:
        for penalty in penalties:
            for c in cs:
                if ((solver == "lbfgs")|(solver == "newton-cg")|(solver == "sag")) & (penalty == "l1"):
                    continue
                clf = LogisticRegression(C=c, penalty = penalty, solver = solver)
                clf.fit(train_feature,train_label)
                preds = clf.predict(dev_feature)
                p,r,f,_ = evaluate_result(dev_label, preds)
                f_score.append(f)
                precision.append(p)
                recall.append(r)
                parameter_save.append((c,penalty,solver))
    i = f_score.index(max(f_score))
    print(precision[i])
    print(recall[i])
    print(f_score[i])
    (c,penalty,solver) = parameter_save[i]
    clf = LogisticRegression(C=c, penalty = penalty, solver = solver)
    clf.fit(train_feature,train_label)
    test_pres = clf.predict(test_feature)
    return test_pres

from sklearn.naive_bayes import MultinomialNB
def tune_nb(train_feature,train_label,dev_feature,dev_label,test_feature):
    alphas = [0.001,0.01,0.1,0.5,0.53,0.55,0.6,1,1.2,1.3,2,10,50,100]
    f_score = []
    precision = []
    recall = []
    parameter_save = []
    for alpha in alphas:
        clf = MultinomialNB(alpha = alpha)
        clf.fit(train_feature,train_label)
        preds = clf.predict(dev_feature)
        p,r,f,_ = evaluate_result(dev_label, preds)
        f_score.append(f)
        precision.append(p)
        recall.append(r)
        parameter_save.append(alpha)
    i = f_score.index(max(f_score))
    print(precision[i])
    print(recall[i])
    print(f_score[i])
    alpha = parameter_save[i]
    clf = MultinomialNB(alpha = alpha)
    clf.fit(train_feature,train_label)
    test_pres = clf.predict(test_feature)
    return test_pres


from sklearn.svm import SVC
import numpy as np
def tune_svm(train_feature,train_label,dev_feature,dev_label,test_feature):
  kernelList = ['linear','rbf','sigmoid']
  gammaList=np.logspace(-10,1,10)
  f_score = []
  precision = []
  recall = []
  parameter_save = []
  for kernel in kernelList:
    for gamma in gammaList:
      svc = SVC(kernel=kernel,gamma=gamma).fit(train_feature,train_label)
      preds = svc.predict(dev_feature)
      p,r,f,_ = evaluate_result(dev_label, preds)
      print(p)
      print(r)
      print(f)
      f_score.append(f)
      precision.append(p)
      recall.append(r)
      parameter_save.append((kernel,gamma))
  i = f_score.index(max(f_score))
  print(precision[i])
  print(recall[i])
  print(f_score[i])
  (kernel,gamma) = parameter_save[i]
  clf = SVC(kernel=kernel,gamma=gamma)
  clf.fit(train_feature,train_label)
  test_pres = clf.predict(test_feature)
  return test_pres

from sklearn.neighbors import KNeighborsClassifier
def tune_knn(train_feature,train_label,dev_feature,dev_label,test_feature):
  list=['euclidean','manhattan','minkowski']
  KList=range(2,13)
  f_score = []
  precision = []
  recall = []
  parameter_save = []
  for i in list:
    for k in KList:
      knn = KNeighborsClassifier(n_neighbors=k,metric=i).fit(train_feature,train_label)
      preds = knn.predict(dev_feature)
      p,r,f,_ = evaluate_result(dev_label, preds)
      print(p)
      print(r)
      print(f)
      f_score.append(f)
      precision.append(p)
      recall.append(r)
      parameter_save.append((k,i))
  i = f_score.index(max(f_score))
  print(precision[i])
  print(recall[i])
  print(f_score[i])
  (k,distance) = parameter_save[i]
  clf = KNeighborsClassifier(n_neighbors=k,metric=distance)
  clf.fit(train_feature,train_label)
  test_pres = clf.predict(test_feature)
  return test_pres

from sklearn import tree
def tune_decisiontree(train_feature,train_label,dev_feature,dev_label,test_feature):
  depthList=np.arange(3,10)
  f_score = []
  precision = []
  recall = []
  parameter_save = []
  for i in depthList:
    dtc = tree.DecisionTreeClassifier(max_depth=i).fit(train_feature,train_label)
    preds = dtc.predict(dev_feature)
    p,r,f,_ = evaluate_result(dev_label, preds)
    print(p)
    print(r)
    print(f)
    f_score.append(f)
    precision.append(p)
    recall.append(r)
    parameter_save.append(i)
  i = f_score.index(max(f_score))
  print(precision[i])
  print(recall[i])
  print(f_score[i])
  depth = parameter_save[i]
  clf = tree.DecisionTreeClassifier(max_depth=depth)
  clf.fit(train_feature,train_label)
  test_pres = clf.predict(test_feature)
  return test_pres

from sklearn.ensemble import RandomForestClassifier
def tune_randomforest(train_feature,train_label,dev_feature,dev_label,test_feature):
  f_score = []
  precision = []
  recall = []
  parameter_save = []
  for i in range(10,101):
    rfc = RandomForestClassifier(n_estimators=i,random_state=13)
    rfc.fit(train_feature,train_label)
    preds = rfc.predict(dev_feature)
    p,r,f,_ = evaluate_result(dev_label, preds)
    print(p)
    print(r)
    print(f)
    f_score.append(f)
    precision.append(p)
    recall.append(r)
    parameter_save.append(i)
  i = f_score.index(max(f_score))
  print(precision[i])
  print(recall[i])
  print(f_score[i])
  n = parameter_save[i]
  clf = RandomForestClassifier(n_estimators=n,random_state=13)
  clf.fit(train_feature,train_label)
  test_pres = clf.predict(test_feature)
  return test_pres

from sklearn.neural_network import MLPClassifier
def tune_mlp(train_feature,train_label,dev_feature,dev_label,test_feature):
  f_score = []
  precision = []
  recall = []
  parameter_save = []
  activations = ["identity","logistic","tanh","relu"]
  solvers = ["lbfgs","sgd","adam"]
  alphas = [1e-4,1e-3,1e-2,0.1,1]
  for activation in activations:
    for solver in solvers:
      for alpha in alphas:
        mlp = MLPClassifier(solver=solver, activation=activation,alpha=alpha,hidden_layer_sizes=(100,), random_state=1,max_iter=100,verbose=10,learning_rate_init=.1)
        mlp.fit(train_feature,train_label)
        preds = mlp.predict(dev_feature)
        p,r,f,_ = evaluate_result(dev_label, preds)
        print(p)
        print(r)
        print(f)
        f_score.append(f)
        precision.append(p)
        recall.append(r)
        parameter_save.append((activation,solver,alpha))
      
  i = f_score.index(max(f_score))
  print(precision[i])
  print(recall[i])
  print(f_score[i])
  (activation,solver,alpha) = parameter_save[i]
  clf = MLPClassifier(solver=solver, activation=activation,alpha=alpha,hidden_layer_sizes=(100,), random_state=1,max_iter=100,verbose=10,learning_rate_init=.1)
  print(clf)
  clf.fit(train_feature,train_label)
  test_pres = clf.predict(test_feature)
  return test_pres

# from xgboost import XGBClassifier
# import numpy as np
# # fit model no training data
# #to be tuned:max_depth、n_estimate、学习率
# def tune_xgb(train_feature,train_label,dev_feature,dev_label,test_feature):
#   depthList=np.arange(3,10)
#   learn_rates = [0.1,0.2,0.3,0.4,0.5]
#   f_score = []
#   precision = []
#   recall = []
#   parameter_save = []
#   for n_estimate in range(10,101):
#     for depth in depthList:
#       for learningrate in learn_rates:
#         model = XGBClassifier(n_estimatores = n_estimate,max_depth = depth,learning_rate = learningrate)
#         model.fit(train_feature, train_label)
#         preds = model.predict(dev_feature)
#         p,r,f,_ = evaluate_result(dev_label, preds)
#         print(p)
#         print(r)
#         print(f)
#         f_score.append(f)
#         precision.append(p)
#         recall.append(r)
#         parameter_save.append((n_estimate,depth,learningrate))
#   i = f_score.index(max(f_score))
#   print(precision[i])
#   print(recall[i])
#   print(f_score[i])
#   (n,d,l) = parameter_save[i]
#   clf = XGBClassifier(n_estimatores = n,max_depth = d,learning_rate = l)
#   clf.fit(train_feature,train_label)
#   test_pres = clf.predict(test_feature)
#   return test_pre

In [51]:
pred_log = tune_log(vector_train,train_labels,vector_development,dev_labels,vector_test) # 0.66
pred_nb = tune_nb(vector_train,train_labels,vector_development,dev_labels,vector_test) #0.61

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

0.8181818181818182
0.9
0.8571428571428572
0.7543859649122807
0.86
0.8037383177570094


In [0]:
pred_log_w2v = tune_log(tfidf_train_vec_skip_gram,train_labels,tfidf_dev_vec_skip_gram,dev_labels,tfidf_test_vec_skip_gram) #to be submitted runned

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACH

In [78]:
pred_log_tfidf = tune_log(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector) #0.68
pred_nb_tfidf = tune_nb(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector) #to be submitted runned

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

0.8846153846153846
0.92
0.9019607843137256
0.7547169811320755
0.8
0.7766990291262137


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
transfer_output(pred_log_tfidf)

In [0]:
pred_log_gw2v = tune_log(X_train_word_average,train_labels,X_dev_word_average,dev_labels,X_test_word_average) #to be submitted runned

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

In [81]:
pred_svm_tfidf = tune_svm(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector)

0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846
0.8518518518518519
0.92
0.8846153846153846


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.803921568627451
0.82
0.8118811881188118
0.8846153846153846
0.92
0.9019607843137256
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.85
0.68
0.7555555555555556
0.8653846153846154
0.9
0.8823529411764707
0.7796610169491526
0.92
0.8440366972477064
0.8846153846153846
0.92
0.9019607843137256


In [21]:
pred_knn_tfidf = tune_knn(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector)

0.8157894736842105
0.62
0.7045454545454546
0.7647058823529411
0.78
0.7722772277227723
0.7954545454545454
0.7
0.7446808510638298
0.7708333333333334
0.74
0.7551020408163266
0.7872340425531915
0.74
0.7628865979381443
0.7755102040816326
0.76
0.7676767676767676
0.782608695652174
0.72
0.7499999999999999
0.7708333333333334
0.74
0.7551020408163266
0.7872340425531915
0.74
0.7628865979381443
0.76
0.76
0.76
0.7551020408163265
0.74
0.7474747474747474
0.5319148936170213
1.0
0.6944444444444444
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.5
1.0
0.6666666666666666
0.8157894736842105
0.62
0.7045454545454546
0.7647058823529411
0.78
0.7722772277227723
0.7954545454545454
0.7
0.7446808510638298
0.7708333333333334
0.74
0.7551020408163266
0.7872340425531915
0.74
0.7628865979381443
0.7755102040816326
0.76
0.76767

In [22]:
pred_decisiontree_tfidf = tune_decisiontree(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector)

0.5208333333333334
1.0
0.684931506849315
0.6024096385542169
1.0
0.7518796992481204
0.6024096385542169
1.0
0.7518796992481204
0.625
1.0
0.7692307692307693
0.6329113924050633
1.0
0.7751937984496124
0.6410256410256411
1.0
0.7812500000000001
0.6363636363636364
0.98
0.7716535433070867
0.6410256410256411
1.0
0.7812500000000001


In [23]:
pred_randomforest_tfidf = tune_randomforest(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector)

0.7090909090909091
0.78
0.7428571428571428
0.7213114754098361
0.88
0.7927927927927928
0.7241379310344828
0.84
0.7777777777777777
0.7049180327868853
0.86
0.7747747747747747
0.7321428571428571
0.82
0.7735849056603773
0.6949152542372882
0.82
0.7522935779816514
0.7192982456140351
0.82
0.7663551401869159
0.6949152542372882
0.82
0.7522935779816514
0.7192982456140351
0.82
0.7663551401869159
0.7068965517241379
0.82
0.7592592592592593
0.7192982456140351
0.82
0.7663551401869159
0.7068965517241379
0.82
0.7592592592592593
0.7321428571428571
0.82
0.7735849056603773
0.7192982456140351
0.82
0.7663551401869159
0.7321428571428571
0.82
0.7735849056603773
0.7321428571428571
0.82
0.7735849056603773
0.7321428571428571
0.82
0.7735849056603773
0.7321428571428571
0.82
0.7735849056603773
0.7321428571428571
0.82
0.7735849056603773
0.7068965517241379
0.82
0.7592592592592593
0.7592592592592593
0.82
0.7884615384615384
0.7454545454545455
0.82
0.780952380952381
0.7592592592592593
0.82
0.7884615384615384
0.7321428571

In [27]:
pred_mlp_tfidf = tune_mlp(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector)

0.8936170212765957
0.84
0.8659793814432989
0.9130434782608695
0.84
0.8749999999999999
0.8979591836734694
0.88
0.888888888888889
0.8518518518518519
0.92
0.8846153846153846
0.8679245283018868
0.92
0.8932038834951457
Iteration 1, loss = 0.64914262
Iteration 2, loss = 0.57880381
Iteration 3, loss = 0.46835045
Iteration 4, loss = 0.33324788
Iteration 5, loss = 0.22961332
Iteration 6, loss = 0.16901454
Iteration 7, loss = 0.13203234
Iteration 8, loss = 0.10648177
Iteration 9, loss = 0.08807138
Iteration 10, loss = 0.07446773
Iteration 11, loss = 0.06310193
Iteration 12, loss = 0.05513173
Iteration 13, loss = 0.04780408
Iteration 14, loss = 0.04162781
Iteration 15, loss = 0.03684286
Iteration 16, loss = 0.03299014
Iteration 17, loss = 0.02964879
Iteration 18, loss = 0.02694449
Iteration 19, loss = 0.02428382
Iteration 20, loss = 0.02207882
Iteration 21, loss = 0.02026603
Iteration 22, loss = 0.01851880
Iteration 23, loss = 0.01715021
Iteration 24, loss = 0.01608571
Iteration 25, loss = 0.0148

Iteration 36, loss = 0.07994069
Iteration 37, loss = 0.07933808
Iteration 38, loss = 0.07880061
Iteration 39, loss = 0.07829529
Iteration 40, loss = 0.07779623
Iteration 41, loss = 0.07725850
Iteration 42, loss = 0.07672049
Iteration 43, loss = 0.07626240
Iteration 44, loss = 0.07577707
Iteration 45, loss = 0.07529935
Iteration 46, loss = 0.07487066
Iteration 47, loss = 0.07444101
Iteration 48, loss = 0.07399367
Iteration 49, loss = 0.07356892
Iteration 50, loss = 0.07313713
Iteration 51, loss = 0.07276498
Iteration 52, loss = 0.07237567
Iteration 53, loss = 0.07203216
Iteration 54, loss = 0.07159137
Iteration 55, loss = 0.07125066
Iteration 56, loss = 0.07089189
Iteration 57, loss = 0.07053617
Iteration 58, loss = 0.07017278
Iteration 59, loss = 0.06982410
Iteration 60, loss = 0.06952353
Iteration 61, loss = 0.06916720
Iteration 62, loss = 0.06887084
Iteration 63, loss = 0.06854192
Iteration 64, loss = 0.06824595
Iteration 65, loss = 0.06798325
Iteration 66, loss = 0.06766757
Iteratio

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 1.14316695
Iteration 2, loss = 1.02540158
Iteration 3, loss = 0.88503880
Iteration 4, loss = 0.73504901
Iteration 5, loss = 0.61351986
Iteration 6, loss = 0.53392030
Iteration 7, loss = 0.48018366
Iteration 8, loss = 0.43989797
Iteration 9, loss = 0.40885534
Iteration 10, loss = 0.38453698
Iteration 11, loss = 0.36399385
Iteration 12, loss = 0.34782239
Iteration 13, loss = 0.33397763
Iteration 14, loss = 0.32242213
Iteration 15, loss = 0.31282948
Iteration 16, loss = 0.30521440
Iteration 17, loss = 0.29831066
Iteration 18, loss = 0.29355486
Iteration 19, loss = 0.28828826
Iteration 20, loss = 0.28435300
Iteration 21, loss = 0.28137484
Iteration 22, loss = 0.27804413
Iteration 23, loss = 0.27592349
Iteration 24, loss = 0.27404370
Iteration 25, loss = 0.27284239
Iteration 26, loss = 0.27077028
Iteration 27, loss = 0.26963928
Iteration 28, loss = 0.26933916
Iteration 29, loss = 0.26791118
Iteration 30, loss = 0.26732003
Iteration 31, loss = 0.26652830
Iteration 32, los

0.8846153846153846
0.92
0.9019607843137256
Iteration 1, loss = 0.66204324
Iteration 2, loss = 0.66064167
Iteration 3, loss = 0.65539236
Iteration 4, loss = 0.65921821
Iteration 5, loss = 0.65412320
Iteration 6, loss = 0.64931997
Iteration 7, loss = 0.64707181
Iteration 8, loss = 0.64074350
Iteration 9, loss = 0.63095856
Iteration 10, loss = 0.62234904
Iteration 11, loss = 0.60733455
Iteration 12, loss = 0.59093276
Iteration 13, loss = 0.56538243
Iteration 14, loss = 0.53800106
Iteration 15, loss = 0.50924033
Iteration 16, loss = 0.46675880
Iteration 17, loss = 0.43351235
Iteration 18, loss = 0.39347614
Iteration 19, loss = 0.35717183
Iteration 20, loss = 0.32331476
Iteration 21, loss = 0.29525667
Iteration 22, loss = 0.26807394
Iteration 23, loss = 0.24592776
Iteration 24, loss = 0.22666324
Iteration 25, loss = 0.21014585
Iteration 26, loss = 0.19455183
Iteration 27, loss = 0.18091009
Iteration 28, loss = 0.17035808
Iteration 29, loss = 0.15912389
Iteration 30, loss = 0.14960483
Iterat

...

Iteration 1, loss = 0.65979674
Iteration 2, loss = 0.62127767
Iteration 3, loss = 0.55621997
Iteration 4, loss = 0.44390416
Iteration 5, loss = 0.31294343
Iteration 6, loss = 0.21727634
Iteration 7, loss = 0.16054652
Iteration 8, loss = 0.12491550
Iteration 9, loss = 0.10103546
Iteration 10, loss = 0.08408106
Iteration 11, loss = 0.07046553
Iteration 12, loss = 0.06089496
Iteration 13, loss = 0.05239448
Iteration 14, loss = 0.04535601
Iteration 15, loss = 0.03988692
Iteration 16, loss = 0.03553368
Iteration 17, loss = 0.03177358
Iteration 18, loss = 0.02879665
Iteration 19, loss = 0.02581877
Iteration 20, loss = 0.02340205
Iteration 21, loss = 0.02141179
Iteration 22, loss = 0.01950007
Iteration 23, loss = 0.01800273
Iteration 24, loss = 0.01684625
Iteration 25, loss = 0.01547657
Iteration 26, loss = 0.01439900
Iteration 27, loss = 0.01347822
Iteration 28, loss = 0.01265576
Iteration 29, loss = 0.01191274
Iteration 30, loss = 0.01124418
Iteration 31, loss = 0.01060911
Iteration 32, los

Iteration 41, loss = 0.07740381
Iteration 42, loss = 0.07685558
Iteration 43, loss = 0.07639387
Iteration 44, loss = 0.07590525
Iteration 45, loss = 0.07542196
Iteration 46, loss = 0.07499064
Iteration 47, loss = 0.07455682
Iteration 48, loss = 0.07410629
Iteration 49, loss = 0.07367772
Iteration 50, loss = 0.07324182
Iteration 51, loss = 0.07286713
Iteration 52, loss = 0.07247594
Iteration 53, loss = 0.07213140
Iteration 54, loss = 0.07168625
Iteration 55, loss = 0.07134397
Iteration 56, loss = 0.07098388
Iteration 57, loss = 0.07062637
Iteration 58, loss = 0.07026082
Iteration 59, loss = 0.06990932
Iteration 60, loss = 0.06960789
Iteration 61, loss = 0.06924986
Iteration 62, loss = 0.06895265
Iteration 63, loss = 0.06862234
Iteration 64, loss = 0.06832466
Iteration 65, loss = 0.06805955
Iteration 66, loss = 0.06774646
Iteration 67, loss = 0.06738362
Iteration 68, loss = 0.06713127
Iteration 69, loss = 0.06681555
Iteration 70, loss = 0.06653388
Iteration 71, loss = 0.06626594
Iteratio

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 1.15329159
Iteration 2, loss = 1.06093504
Iteration 3, loss = 0.95259391
Iteration 4, loss = 0.82302767
Iteration 5, loss = 0.68471248
Iteration 6, loss = 0.57352344
Iteration 7, loss = 0.49988020
Iteration 8, loss = 0.45033503
Iteration 9, loss = 0.41505552
Iteration 10, loss = 0.38852220
Iteration 11, loss = 0.36671587
Iteration 12, loss = 0.34971330
Iteration 13, loss = 0.33534973
Iteration 14, loss = 0.32344824
Iteration 15, loss = 0.31361627
Iteration 16, loss = 0.30583059
Iteration 17, loss = 0.29880346
Iteration 18, loss = 0.29396611
Iteration 19, loss = 0.28861928
Iteration 20, loss = 0.28463802
Iteration 21, loss = 0.28163557
Iteration 22, loss = 0.27823110
Iteration 23, loss = 0.27609359
Iteration 24, loss = 0.27418330
Iteration 25, loss = 0.27296586
Iteration 26, loss = 0.27089127
Iteration 27, loss = 0.26972607
Iteration 28, loss = 0.26941707
Iteration 29, loss = 0.26800187
Iteration 30, loss = 0.26739827
Iteration 31, loss = 0.26658560
Iteration 32, los

Iteration 6, loss = 0.90668201
Iteration 7, loss = 0.95398497
Iteration 8, loss = 1.07030547
Iteration 9, loss = 0.94064259
Iteration 10, loss = 0.83453796
Iteration 11, loss = 0.74152148
Iteration 12, loss = 0.75199616
Iteration 13, loss = 0.77413046
Iteration 14, loss = 0.74893266
Iteration 15, loss = 0.72340662
Iteration 16, loss = 0.80040964
Iteration 17, loss = 0.79446924
Iteration 18, loss = 0.71624115
Iteration 19, loss = 0.72535628
Iteration 20, loss = 0.71893356
Iteration 21, loss = 0.70949800
Iteration 22, loss = 0.71722435
Iteration 23, loss = 0.74408627
Iteration 24, loss = 0.70766326
Iteration 25, loss = 0.73501941
Iteration 26, loss = 0.72373391
Iteration 27, loss = 0.69915894
Iteration 28, loss = 0.70369103
Iteration 29, loss = 0.76813527
Iteration 30, loss = 0.76452777
Iteration 31, loss = 0.70307174
Iteration 32, loss = 0.67792465
Iteration 33, loss = 0.67654284
Iteration 34, loss = 0.68928619
Iteration 35, loss = 0.68979888
Iteration 36, loss = 0.71475177
Iteration 37

In [29]:
transfer_output(pred_mlp_tfidf)

In [30]:
sum(pred_mlp_tfidf)

206

In [23]:
pred_tfidf_xgb = tune_xgb(train_tfidfvector,train_labels,dev_tfidfvector,dev_labels,test_tfidfvector) #to be submitted runned

KeyboardInterrupt: ignored

In [12]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten

# set parameters:
batch_size = 32
n_filter = 16
filter_length = 4
nb_epoch = 5
n_pool = 2

model = Sequential()
model.add(Convolution2D(n_filter,filter_length,filter_length,
                        input_shape=(1, 256, 128)))
model.add(Activation('relu'))
model.add(Convolution2D(n_filter,filter_length,filter_length))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='mse',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(train_tfidfvector, train_labels, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=0)
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (4, 4), input_shape=(1, 256, 1...)`
  from ipykernel import kernelapp as app


ValueError: ignored

In [0]:
######################################### LDA ##################################################

In [39]:
from sklearn.decomposition import LatentDirichletAllocation
n_topic = 1
lda = LatentDirichletAllocation(n_topics=n_topic,max_iter=2000,learning_method='batch')
one = tfidfvector.fit_transform(preprocessed_train_events[0])
lda.fit(one)

TypeError: ignored